In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
df = pd.read_csv('scraped_data.csv')
df.head(5)

In [ ]:
df['Property Type'].value_counts()


# Data Cleaning

In [ ]:
#removing the link column 
df = df.drop('Link', axis=1)

In [ ]:
#replace all the Not available with NaN
df.replace("Not Available", np.nan, inplace=True)
df.head(5)

In [ ]:
# Checking nan values
# Check for NaN values in each column
nan_counts_per_column = df.isna().sum()
print("NaN counts per column:")
print(nan_counts_per_column)

In [ ]:
# Define a function to fill missing 'Age' values within each group
def fill_age(group):
    # Use the first non-NaN 'Age' value found in each group to fill the NaNs
    return group.fillna(method='ffill').fillna(method='bfill')

# Apply the function to each group for the 'Age' column
grouped = df.groupby('Property Name')
df['Age'] = grouped['Age'].transform(fill_age)


In [ ]:
# Checking nan values
# Check for NaN values in each column
nan_counts_per_column = df.isna().sum()
print("NaN counts per column:")
print(nan_counts_per_column)

In [ ]:
df['PSF'].value_counts()

In [ ]:
# Remove the rows with no addresses
df = df[df['Address'].notna()]
df.head()

In [ ]:
import pandas as pd

def clean_bedroom(value):
    # Convert NaN and float values to string to safely use string operations
    value = str(value)
        
    # Handle 'Studio' entries
    if 'Studio' in value:
        return 1
    # Handle entries like '3 bedroom (Dual Key)' and '2 bedroom (Dual Key)'
    elif 'bedroom' in value:
        return int(value.split()[0])
    # Handle entries with '+' like '3+1'
    elif '+' in value:
        parts = value.split('+')
        return sum(int(part) for part in parts if part.isdigit())
    # Default case, check if it's a digit-only string and convert it to integer
    elif value.isdigit():
        return int(value)
    # Return NaN for non-numeric or unexpected values to avoid data corruption
    else:
        return pd.NA

# Apply the function to the 'Bedrooms' column
df['Bedrooms'] = df['Bedrooms'].apply(clean_bedroom)

In [ ]:
import pandas as pd

def clean_tenure(value):
    # Check if the value is a string
    if pd.isna(value):
        return value  # Keep NaN as is or you can specify a default string if needed
    value = str(value)  # Convert to string to ensure string operations can be applied

    # Handle 'Freehold' entries
    if 'freehold' in value.lower():
        return '9999 years'
    # Extract number of years from 'LEASEHOLD/...' or similar formats
    elif 'leasehold/' in value.lower():
        return value.split('/')[1].strip() + ' years'
    # Handle specific year terms directly provided in the string
    elif 'years' in value.lower():
        # Try to find the number of years explicitly mentioned
        import re
        match = re.search(r'\b(\d+)\s*years', value, re.IGNORECASE)
        if match:
            return match.group(1) + ' years'
    # Handle any remaining strings that may directly start with the number of years
    else:
        import re
        match = re.search(r'^(\d+)', value)
        if match:
            return match.group(1) + ' years'
    # Return the original value if none of the above conditions are met
    return value

# Apply the function to the 'Tenure' column
df['Tenure'] = df['Tenure'].apply(clean_tenure)

import pandas as pd

def clean_and_convert_tenure(value):
    if pd.isna(value):
        return value  # Return NaN as it is
    value = str(value).lower()  # Convert to lowercase string for uniform processing
    if 'years' in value:
        value = value.replace('years', '').strip()  # Remove the word 'years'
    if value.isdigit():
        return int(value)  # Convert to integer
    return value  # Return the original value if it's not a clean number

# Apply the function to the 'Tenure' column to clean and convert it
df['Tenure'] = df['Tenure'].apply(clean_and_convert_tenure)


In [ ]:
df.head()

In [ ]:
# Check for the new nan values
nan_counts_per_column = df.isna().sum()
print("NaN counts per column:")
print(nan_counts_per_column)

In [ ]:
# Drop rows where the 'PSF' column is NaN
df = df.dropna(subset=['PSF'])

In [ ]:
#after removing all the irrelevant data, all the data with no asking price is GONE YAY !
nan_counts_per_column = df.isna().sum()
print("NaN counts per column:")
print(nan_counts_per_column)

In [ ]:
import re
def clean(text):
    return re.sub("\D","",str(text))
df["Size"] = df["Size"].apply(lambda x:clean(x))
df["District"] = df["District"].apply(lambda x:clean(x))
df["Asking Price"] = df["Asking Price"].apply(lambda x:clean(x))

# Transforming categorical variables

In [ ]:
# Changing property types to ints
df["Property Type"].replace({"Apartment":int(0), "Condominium":int(1)}, inplace=True)

In [ ]:
df['Age'] = pd.to_numeric(df['Age']).astype('Int64')

In [ ]:
# Remove rows where 'Tenure' is 'n.a'
df = df[df['Tenure'] != 'n.a']
# Convert 'Tenure' to numeric; all values appear to be strings of year numbers
df['Tenure'] = pd.to_numeric(df['Tenure'])


In [ ]:
df['Years_Left'] = df['Tenure'] - df['Age']

In [ ]:
# Checking that all ammenity elements are dicts
def check_same_type(lst):
    if not lst:
        return False  # Empty list has no type
    first_type = type(lst[0])
    return all(type(item) == first_type for item in lst[1:])

amenities = df['Amenities'].tolist()

print(check_same_type(amenities))

In [ ]:
# Convert each element in the Amenities column to a dict
import ast

def parse_str_to_dict(string_dict):
    parsed_dict = ast.literal_eval(string_dict)
    return parsed_dict

amenities = list(map(lambda element: parse_str_to_dict(element), df['Amenities'].tolist()))

In [ ]:
# Need to rerun all cells for this to work
df_normalized = pd.json_normalize(amenities)
df = df.drop('Amenities', axis=1)
df_normalized.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df, df_normalized], axis=1)

In [ ]:
# How many Primary schools how many sec schools?
df['Primary Schools'] = df['Primary Schools'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df['Secondary Schools'] = df['Secondary Schools'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df['Shopping Malls'] = df['Shopping Malls'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df['Groceries & Supermarts'] = df['Groceries & Supermarts'].apply(lambda x: len(x) if isinstance(x, list) else 0)

df.head(5)


In [ ]:
df["No. of Amenities"] = df['Primary Schools'] + df['Secondary Schools'] + df['Shopping Malls'] + df['Groceries & Supermarts']
df.head(5)

In [ ]:
df['Years_Left'] = df['Tenure'] - df['Age']

In [ ]:
df = df.drop(columns=['No. of Units', 'PSF', 'Tenure'])


In [ ]:
# Convert 'Bedrooms', 'Bathrooms', 'Asking Price', and 'Size' to integer
df['Bedrooms'] = pd.to_numeric(df['Bedrooms']).astype('Int64')
df['Bathrooms'] = pd.to_numeric(df['Bathrooms']).astype('Int64')
df['Asking Price'] = pd.to_numeric(df['Asking Price']).astype('Int64')
df['Size'] = pd.to_numeric(df['Size']).astype('Int64')
df['Age'] = pd.to_numeric(df['Age']).astype('Int64')

In [ ]:
# Checking nan values
# Check for NaN values in each column
nan_counts_per_column = df.isna().sum()
print("NaN counts per column:")
print(nan_counts_per_column)

In [ ]:
df = df.dropna()


In [ ]:
import pandas as pd

# Assuming your data is stored in a Pandas DataFrame named 'df'
# Replace 'df' with the name of your actual DataFrame

# Check the shape of the DataFrame
print("Shape of the DataFrame:", df.shape)

# Check the data types of each column
print("Data types of each column:")
print(df.dtypes)

# Check for missing or NaN values
missing_values = df.isnull().sum()
if missing_values.sum() > 0:
    print("Number of missing values in each column:")
    print(missing_values)
else:
    print("No missing values found.")

# Check for unique values in each column
print("Unique values in each column:")
for column in df.columns:
    unique_values = df[column].unique()
    print(f"{column}: {unique_values}")

# Check summary statistics of the DataFrame
print("Summary statistics:")
print(df.describe())


In [ ]:
df = df.drop(columns=['Property Name'])

In [ ]:
#This will take 10 years to run lol
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Function to apply geocoding
def geocode_address(addr, geocode):
    try:
        location = geocode(f"{addr}, Singapore")  # Force the context to Singapore in the query
        if location and 'Singapore' in location.address:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except:
        return pd.Series([None, None])

# Function to process chunks of addresses
def process_addresses(df, chunk_size=1000):
    # Initialize the geocoder
    geolocator = Nominatim(user_agent="NUS_project")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    # Number of chunks
    num_chunks = (len(df) // chunk_size) + (1 if len(df) % chunk_size != 0 else 0)
    
    for i in range(num_chunks):
        # Define chunk limits
        start_idx = i * chunk_size
        end_idx = start_idx + chunk_size
        
        # Process chunk
        chunk = df.iloc[start_idx:end_idx]
        chunk[['Latitude', 'Longitude']] = chunk['Address'].apply(lambda x: geocode_address(x, geocode))
        
        # Save results to a CSV file
        chunk.to_csv(f'geocoded_addresses_{start_idx}_{end_idx}.csv', index=False)
        
        print(f"Processed and saved chunk {i+1}/{num_chunks} (rows {start_idx} to {end_idx})")

# Assuming df is your DataFrame loaded with addresses 
process_addresses(df)
